In [18]:
# Model training for DRMM
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
from keras.utils.np_utils import to_categorical
from collections import defaultdict
import numpy as np
import make_test_data
import pickle as pkl
import os
import pdb
from sklearn.linear_model import LogisticRegression

#TODO: fix root, remove key restrictions

# Placeholder data parsing
def get_data(histograms, training_year=2014, training=True):
    # Proposed format: [doc_id topic_id h0 h1 h2 h3 ...]
    #data_root = '/scratch/cluster/dnelson/ir_proj'

    # can set this to 2015 to read in annotations for 2015 instead
    # format: label_dict[doc_id][topic_id] = ground truth
    label_dict = make_test_data.make_truth(training_year)

    # enforcing order on a dictionary & downsampling to only data w/judgments for training
    key_array = histograms.keys()
    if training:
        key_array = [val for val in key_array if label_dict[int(val[0])][int(val[1])+1] >= 0]

    X = np.array([[np.log(val) if val > 0 else val for val in histograms[key]] for key in key_array])
    Y = np.array([int(label_dict[int(key[0])][int(key[1])+1] > 0) for key in key_array])
    return X, Y, key_array

def get_dict(training_year=2014):
    data_root = '/Users/Dan/class/deep_ir/project/data'
    
    with open(os.path.join(data_root, 'histograms_%d' % training_year), 'r') as f:
        histograms = pkl.load(f)
    return histograms


def get_fake_data():
    X_train = np.random.rand(1000, 29)

    # roughly uniform dist of 0, 0.5, 1
    rand_vals = np.random.rand(1000)
    Y_train = np.array([np.floor(val * 3)/2.0 for val in rand_vals])
    return X_train, Y_train

In [15]:
# Get all data for maximum iteration funtime power
# THIS IS THE LONG PART
train_dict = get_dict(training_year=2014)
test_dict = get_dict(training_year=2015)

In [19]:
X_train, Y_train, _ = get_data(train_dict, training_year=2014, training=True)
#X_train, Y_train = get_fake_data()
X_test, Y_test, test_keys = get_data(test_dict, training_year=2015, training=False)

qrels2014.txt
qrels-treceval-2015.txt


In [57]:
# ASIDE - this is how you make your own loss function. Not necessary but fun to play with.
from keras import backend as K

def supercool_loss(y_true, y_pred):
    return K.mean(K.abs(y_true*10 - y_pred))
    

In [60]:
# Create model (input_shape is inferred after first layer)
# This model is a regression
model = Sequential()
model.add(Dense(output_dim=29, input_shape=(29,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(output_dim=5))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(output_dim=1, activation='hard_sigmoid'))

# Compile model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True, clipnorm=1.)
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['binary_accuracy'])

# Train model
model.fit(X_train, Y_train, nb_epoch=5, batch_size=8)

Epoch 1/5
31403/31403 [==============================] - 16s - loss: 1.3158 - binary_accuracy: 0.9057    
Epoch 2/5
31403/31403 [==============================] - 16s - loss: 1.3266 - binary_accuracy: 0.9075    
Epoch 3/5
25520/31403 [=======================>......] - ETA: 3s - loss: 1.3385 - binary_accuracy: 0.9067

KeyboardInterrupt: 

In [51]:
from keras.utils.np_utils import to_categorical

categorical_labels = to_categorical(Y_train, nb_classes=None)
print categorical_labels[:10]

[[ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]]


In [44]:
# Test model
pred_ranks = model.predict(X_test)
print max(pred_ranks)

[ 0.22982793]


In [46]:


#[0][0] = id, [0][1] = topic, [1] = rank
ranks_and_keys = zip(test_keys, pred_ranks)
result_dict = defaultdict(lambda: [])
for val in ranks_and_keys:
    result_dict[val[0][1]].append((val[1][0], val[0][0]))

with open('cat_query_results','wb') as f:
    for cur_topic in range(30):
        topic_results = result_dict[str(cur_topic)]
        topic_results.sort(reverse=True)

        for result_rank, cur_result in enumerate(topic_results[:1000]):
            line_to_write = [str(cur_topic + 1), '0', str(cur_result[1]), str(result_rank + 1), str(cur_result[0]), 'test_run', '\n']
            f.write(" ".join(line_to_write))

In [45]:
print max(pred_ranks)

[ 0.22982793]


In [8]:
print int(True)

1
